In [3]:
import pandas as pd
post = pd.read_csv('posts.sample.csv')
post.head(5)
post.describe()

,PostTypeId,AcceptedAnswerId,Score,ViewCount,OwnerUserId,LastEditorUserId,AnswerCount,CommentCount,FavoriteCount
count,102.000000,25.000000,102.000000,28.000000,102.000000,54.000000,28.000000,102.000000,28.000000
mean,1.725490,659.880000,29.107843,32923.750000,460.225490,8430.962963,7.035714,4.117647,15.107143
std,0.448471,2215.389588,41.985124,50467.911432,2127.383699,19853.439665,4.122945,4.115400,19.120164
min,1.000000,12.000000,-1.000000,538.000000,5.000000,-1.000000,2.000000,0.000000,1.000000
25%,1.000000,47.000000,6.250000,3914.000000,23.000000,49.500000,3.750000,1.000000,3.750000
50%,2.000000,87.000000,14.500000,17243.000000,39.000000,300.000000,6.500000,3.000000,7.000000
75%,2.000000,127.000000,29.750000,38666.250000,67.000000,9032.250000,9.250000,6.000000,15.500000
max,2.000000,10600.000000,250.000000,228041.000000,13078.000000,102286.000000,20.000000,21.000000,86.000000


In [2]:
comments = pd.read_csv('comments.sample.csv')
comments.head(5)
comments.describe()

,Id,PostId,Score,UserId
count,102.000000,102.000000,102.000000,96.000000
mean,69.284314,72.068627,3.931373,52.020833
std,40.807779,51.506082,6.124549,26.968980
min,2.000000,1.000000,0.000000,6.000000
25%,33.250000,29.000000,0.000000,28.250000
50%,68.500000,51.000000,2.000000,45.000000
75%,102.750000,127.000000,5.000000,78.000000
max,140.000000,158.000000,40.000000,101.000000
